In [5]:
import numpy as np
import scipy.stats

# Optimally Solving the Bandit Problem

- since the bandit problem has a small finite horizon, you can work backwards optimally solve the problem. (otherwise you'd have to use the gittens index or something to optimally balance exploration and exploitation) 
- but you haven't seen any options? You can enumerate each possibility at each stage? A branching tree and then back-track. So the first stage as 8 possible outcomes. 4 choices, 2 outcomes each. Or do we want a (state,action) pair. 
- This is similar to how you would solve with dynamic programming. Wait is this dynamic programming on belief state POMDP? I think it might be. Ok. step 1. enumerate the tree (but for state or actions). Just the possibilities, or the beliefs as well. step 2: start at the end, which branch? and say ok I chose bandit 1.

- state's might be belief states. S = [2,3; 1,2; 0,1] after six trials. 
- the we also have the expected total reward to be followed on the remaining trials. u(S)?
    - s+A / s+f+a+b * u(k+1, where s+1)  + f+B / s+f+a+b * u(k+1, where f+1)
- you then choose the max. of this. 



Dynamic programming: 

In the value iteration, you loop through the states (in no particular order) and increment the value function for each state based on next set of states weighted by their probabilities. 

In this case, we can start at the end state instead of sampling states randomly. Therefore, we only need to do one pass through the states. 

We update the value function for each state, based on the probability of going to the next state, which is given by what we've observed in each state, and then update it's value based on what we find there. 

Then to actually do the task, we simply solve forward by choosing the best action. 

We need an efficient and intuitive state representation and implementation in code. 
What's important about the state: 
- number of heads, tails for each option in order .. option1, option2 etc. (I'll try this first with just a few states). 
- or number of heads, tails for best option, second best option etc. 



states tensor = number heads X number tails X option # not all of this is possible at once? .. nevermind. 


using the full bayesian solution is called solving the bandit process - gittins provides a simpler calculation. https://people.eecs.berkeley.edu/~avivt/BRLS_journal.pdf




In [6]:
def bandit_dp(K=2,N=5,alpha=1.0,beta=1.0):

    # value function, indexed by state 
    v = np.empty((np.repeat(N,K*2)))*np.nan   # ***might need to change this to Q values # 
    q = np.empty((np.append(np.repeat(N,K*2),np.array([K]))))*np.nan   # ***might need to change this to Q values # 
    
    # state representations = [s1,f1,s2,f2,...sk,fk]
    # some states are impossible, but they will stay nan's. 

    for trial in np.arange(N)[::-1]:
        #print('')
        print('trial {0}').format(trial)

        # enumerate each possible state
        for state,__ in np.ndenumerate(v):
            state = np.array(state) 

            # only consider states for this trial with 
            if np.sum(state)==(trial): 
                #print('state:{0}').format(state)
                for bandit in range(K):
                    #
                    #print('bandit: {0}').format(bandit)
                    s = np.float(state[0+bandit*K])
                    f = np.float(state[1+bandit*K])

                    if trial==N-1: # last trials
                        # final state value 
                        vnext_s=0.0
                        vnext_f=0.0
                    else:
                        # increment the success for that bandit to get next state
                        snext_s = state.copy()
                        snext_s[0+bandit*K]+=1 
                        vnext_s = v[tuple(snext_s)]

                        # increment the failure for that bandit to get next state
                        snext_f = state.copy()
                        snext_f[1+bandit*K]+=1  
                        vnext_f = v[tuple(snext_f)]

#                         print('snext_success: {0}').format(snext_s)
#                         print('snext_fail: {0}').format(snext_f)
#                     print('qnext_fail: {0}').format(qnext_f)
#                     print('qnext_success: {0}').format(qnext_s)
                    es = (s+alpha)/(s+f+alpha+beta)
                    ef = (f+beta)/(s+f+alpha+beta)
                    qval = es*(vnext_s+1.0) + ef*vnext_f
#                     print('weight s: {0}').format(es)
#                     print('weight f: {0}').format(ef)
#                     print('q: {0}').format(qval)
                    q[tuple(np.append(state,bandit))]=qval

                # take max value over actions in this state
                v[tuple(state)]=np.max(q[tuple(state)])
    return(v,q)



In [7]:
## simple problem with 2 bandits, 3 trials. 
v,q = bandit_dp(K=2,N=4,alpha=1.0,beta=1.0)

print('')
print('finished results:')
print('first trial values')
print(q[(0,1,0,0)]) # one failure, you're expected value is still 0.5, because you choose that one 
print(q[(1,0,0,0)]) # one success, you're expected value is 0.666, because you choose the next one. 


print('')
print('2nd trial')
print(q[(0,1,0,1)]) # one failure for each. and you have one choice left, what's you're expected values? 
print(q[(1,0,1,0)]) # one success for each: 
print(q[(1,0,0,1)]) # one success, one failure: notice how it's the same as 2 success, because you are assumed to choose the best. 
print(q[(2,0,0,0)]) # two successes
print(q[(0,2,0,0)]) # two failes


print('')
print('3rd trial')
print(q[(0,2,0,1)]) # two failures, one failure and you have one choice left, what's you're expected values? 
print(q[(2,0,1,0)]) # two successes, one success
print(q[(2,0,0,1)]) # two successes, one failure
print(q[(2,1,0,0)]) # two/1
print(q[(3,0,0,0)]) # 


trial 3
trial 2
trial 1
trial 0

finished results:
first trial values
[ 1.36111111  1.52777778]
[ 2.02777778  1.86111111]

2nd trial
[ 0.72222222  0.72222222]
[ 1.38888889  1.38888889]
[ 1.33333333  1.        ]
[ 1.5   1.25]
[ 0.75  1.  ]

3rd trial
[ 0.25        0.33333333]
[ 0.75        0.66666667]
[ 0.75        0.33333333]
[ 0.6  0.5]
[ 0.8  0.5]


In [9]:
### play bandit game #### 

# specify game 
K = 2
N = 20

# specify starting belief
alpha=1.0
beta=1.0

# solve for the optimal value function for each possible state of the game 
print('solving game...')
v,q = bandit_dp(K,N,alpha,beta)

# specify starting state, and bandit probabilities 
s0 = np.array([0,0,0,0])
p = [0.3,0.7]

#
print('playing game...')

for trial in np.arange(N): #[0,1,2]: #np.arange(N):
    print('trials: {0}').format(trial)
    if trial==0:
        state = s0
    
    # choose action # 
    qvals = q[tuple(state)]
    bandit = np.argmax(qvals)
    
    # observe response 
    outcome = scipy.stats.bernoulli(p[bandit]).rvs() # different for different bandits # 
    
    print('state: {0}').format(state)
    print('q: {0}').format(qvals)
    print('action: {0}').format(bandit)
    print('outcome: {0}').format(outcome)
    print('')
    

    # increment state
    state[(1-outcome)+bandit*K]+=1
    


solving game...
trial 19
trial 18
trial 17
trial 16
trial 15
trial 14
trial 13
trial 12
trial 11
trial 10
trial 9
trial 8
trial 7
trial 6
trial 5
trial 4
trial 3
trial 2
trial 1
trial 0
playing game...
trials: 0
state: [0 0 0 0]
q: [ 12.43126491  12.43126491]
action: 0
outcome: 1

trials: 1
state: [1 0 0 0]
q: [ 13.51116571  13.39216399]
action: 0
outcome: 1

trials: 2
state: [2 0 0 0]
q: [ 13.82476132  13.64625874]
action: 0
outcome: 1

trials: 3
state: [3 0 0 0]
q: [ 13.7265145   13.48851106]
action: 0
outcome: 1

trials: 4
state: [4 0 0 0]
q: [ 13.38162552  13.0904842 ]
action: 0
outcome: 1

trials: 5
state: [5 0 0 0]
q: [ 12.87383719  12.54202475]
action: 0
outcome: 0

trials: 6
state: [5 1 0 0]
q: [ 10.58577281  10.42925321]
action: 0
outcome: 0

trials: 7
state: [5 2 0 0]
q: [ 8.90847813  8.9128011 ]
action: 1
outcome: 1

trials: 8
state: [5 2 1 0]
q: [ 8.7524631   8.82395189]
action: 1
outcome: 1

trials: 9
state: [5 2 2 0]
q: [ 8.43423035  8.54146354]
action: 1
outcome: 1

tria